# Introduction to Python for Earth Scientists

# 1. Introduction to programming, Python, and Jupyter notebooks
## A.
- Why program?
- Why Python?
- Jupyter notebooks
- Good practice and documentation
- Finding help

## B.
- Basic data types
- Variables
- Getting data
- Basic plotting

# 2. Loading, viewing and plotting data
## A.
- Matplotlib
- Pandas
- Looping  
- Simple functions
- Saving plots

## B.
- Numpy
- Linear regression

# 3. More complex plotting
## A.
- Slicing and dicing dataframes
- Subplots
- Looping over subplots

## B.
- Time series
- Formatting dates and times
- Moving averages 
- Applying functions to dataframes

# 4. Making maps
## A.
- Cartopy
- Projections
- Plotting point data
- Annotating maps

## B.
- Contouring
- Mapping with vectors

The content may change in response to students' questions or current events. Each of the four modules has been designed to take about three hours, with a short break between each of the two parts.

# What, why and how do we programme?

<img alt="xkcd Compilation" align="right" style="width:50%" src="https://imgs.xkcd.com/comics/compiling.png">

1. Why programming;
2. Earth Science data: seismology-bias;
3. Why Python;
4. About the author;
5. How to use these notebooks;

## Why programming (what's wrong with Excel!?)

**Reproducibility**,  **Safety**,  **Speed**,  **Complexity**,  **Data scale** 
    
<img alt="XKCD spreadsheets" align="center" style="width:80%" src="https://imgs.xkcd.com/comics/spreadsheets.png">

## Why programming (what's wrong with Excel!?)

- **Reproducibility:** If someone can't replicate your 
  work, why should we trust it to be true?
- **Safety:** Your data and your processing should not
  overlap.  Your raw data should be sacred.
- **Speed:** You want a result, and you want it yesterday... Learn how to write good code 
    (and change the clock on your computer) and you can...
- **Complexity:** Being able to solve complex problems logically, in a way that others can follow
    (and reproduce) is essential to natural sciences. *Hint: Writing good code is as much about the*
    *quality of your documentation as it is about the quality of your code*.
- **Data scale:** Data in natural sciences is noisy, and large. Ideally to understand the natural world
    we would have data from every place at every time throughout the Earth. We don't have that, but
    our datasets are growing...

## An example of dealing with lots of data
Seismographs deployed for:
- long durations (multi-year);
- multiple locations;
- modest sampling rates.

For example: [SAMBA](http://ds.iris.edu/mda/9F/?starttime=2008-01-01T00:00:00&endtime=2020-12-31T23:59:59) is
the Southern Alps Microearthquake Borehole Array around Mt. Cook has been recording since 2008.

<img alt="SAMBA" align="right" style="width:100%" src="images/COVA_pano.jpg">

SAMBA records at 200Hz (200 samples per second). How many seconds per day?

In [26]:
seconds_per_day = 60 * 60 * 24
print(f"There are {seconds_per_day} seconds in a day")

There are 86400 seconds in a day


How many samples per day?

In [27]:
sampling_rate = 200.0
samples_per_day = seconds_per_day * sampling_rate
print(f"SAMBA records {samples_per_day} samples per day")

SAMBA records 17280000.0 samples per day


So, > 17 million samples per day.  But that is just for one channel: SAMBA seismographs
have three channels, a vertical and two horizontals, so how many samples per day for one station?

In [28]:
number_of_channels = 3
samples_per_day_per_station = samples_per_day * number_of_channels
print(f"One station records {samples_per_day_per_station} samples per day")

One station records 51840000.0 samples per day


Nearly 52 million samples per day per station. SAMBA is made up of 13 stations, so our dataset gets bigger
still:

In [29]:
number_of_stations = 13
samples_per_day_total = samples_per_day_per_station * number_of_stations
print(f"SAMBA records {samples_per_day_total} samples per day")

SAMBA records 673920000.0 samples per day


673 million samples per day across the network. So what is that over the first 10 years of operation?

In [30]:
days_per_year = 365.25 # Roughly
samples_per_year = days_per_year * samples_per_day_total
print(f"SAMBA records about {samples_per_year} samples per year.")
samples_per_decade = samples_per_year * 10
print(f"In 10 years of operation SAMBA recorded something like {samples_per_decade:e} samples")

SAMBA records about 246149280000.0 samples per year.
In 10 years of operation SAMBA recorded something like 2.461493e+12 samples


2 Trillion samples.

Try working with that in a spreadsheet...

## Why Python?

Our first notebook covers this a little more, but the key things for me are:

1. Open-source, community-driven (free) software;
2. Simple syntax, fast to make mistakes and helpful error messages;
3. Community libraries to do lots of complex tasks 
   (e.g. [obspy](https://github.com/obspy/obspy/wiki) for seismology)

# What are Jupyter notebooks and how do we use them?

There are a series of Jupyter notebooks on here.  You can run them interactively in your
browser.  You should run through them, change some values, see what works, try and play
with variables and experiment.  There will be sections that you are expected to fill in
marked as **Exercise:**.  Shout out if and when you have problems.

Let us know if you want to play around with any other data.

Remember that this course is supposed to be a brief look over some of the key ideas in Python
and useful libraries, it is not complete!

## Getting set-up on a CO501 computer
1. Log into your Linux account
2. Open a terminals and set-up your conda virtual environment
```bash
startconda
conda activate esci451
```
3. Clone the notebook repository
```bash
git clone https://github.com/calum-chamberlain/ESCI451-Python.git
```
4. Start Jupyter:
```bash
cd ESCI451-Python
jupyter notebook
```